In [107]:
import pandas as pd
import numpy as np
import random
import math
import re


In [108]:
df = pd.read_csv("tourism_with_id.csv", encoding='ISO-8859-1')
df = df[df["City"] == "Jakarta"]
price_values = df['Price(RPH)'].tolist()

for i in range(len(price_values)):
  converted_amount = price_values[i] * 0.0054
  df['Price(RPH)'][i] = math.ceil(converted_amount)

df.rename(columns={'Price(RPH)': 'Price (INR)'}, inplace=True)

category_avg = df.groupby('Categories')['Time_Minutes'].mean()

# Replace NaN values with the average time taken for each category
df['Time_Minutes'] = df.apply(lambda row: category_avg[row['Categories']] if np.isnan(row['Time_Minutes']) else row['Time_Minutes'], axis=1)
is_nan_present_in_column = df['Time_Minutes'].isna().values.any()
df.loc[df['Categories'] == 'Islands', 'Time_Minutes'] = df.loc[df['Categories'] == 'Islands', 'Time_Minutes'].fillna(90)
df.loc[df['Categories'] == 'Religious', 'Time_Minutes'] = df.loc[df['Categories'] == 'Religious', 'Time_Minutes'].fillna(45)
is_nan_present_in_column = df['Time_Minutes'].isna().values.any()
df['Time_Minutes'] = df['Time_Minutes'].apply(math.ceil)
display(df)

final_df = df

C:\Users\prana\AppData\Local\Temp\ipykernel_2900\3893822602.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Price(RPH)'][i] = math.ceil(converted_amount)


,Place_Id,Place_Name,Description,Categories,City,Price (INR),Ratings,Time_Minutes,Coordinate,lat,Long
0,1,National Monument,The National Monument or popularly abbreviated...,Culture,Jakarta,108,4.6,15,"{'lat': -6.1753924, 'lng': 106.8271528}",-6.175392,106.827153
1,2,Old town,"The old town in Jakarta, which is also called ...",Culture,Jakarta,0,4.6,90,"{'lat': -6.137644799999999, 'lng': 106.8171245}",-6.137645,106.817125
2,3,Fantasy world,Dunia Fantasi or also called Dufan is an enter...,Amusement park,Jakarta,1458,4.6,360,"{'lat': -6.125312399999999, 'lng': 106.8335377}",-6.125312,106.833538
3,4,Beautiful Indonesia in Miniature Park (TMII),Taman Mini Indonesia Indah is a tourist park w...,Amusement park,Jakarta,54,4.5,134,"{'lat': -6.30245899999999, 'lng': 106.8951559}",-6.302446,106.895156
4,5,Atlantis Water Adventure,"Atlantis Water Adventure, also known as Atlant...",Amusement park,Jakarta,508,4.5,60,"{'lat': -6.12419, 'lng': 106.839134}",-6.124190,106.839134
...,...,...,...,...,...,...,...,...,...,...,...
79,80,Plaza Indonesia,"Plaza Indonesia was inaugurated in early 1990,...",Shopping center,Jakarta,0,4.7,120,"{'lat': -6.193925900000002, 'lng': 106.8222158}",-6.193926,106.822216
80,81,Thamrin City Mall,Thamrin City or Thamrin City Mall is a large s...,Shopping center,Jakarta,0,4.4,60,"{'lat': -6.1946096, 'lng': 106.817905}",-6.194610,106.817905
81,82,The Satria Mandala Museum,The Satria Mandala Museum is a museum of the h...,Culture,Jakarta,27,4.5,150,"{'lat': -6.231698499999999, 'lng': 106.8185425}",-6.231699,106.818543
82,83,Alive Museum Ancol,Museums now don't only offer objects of histor...,Amusement park,Jakarta,1080,4.3,134,"{'lat': -6.117534399999999, 'lng': 106.857313}",-6.117534,106.857313


In [109]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 84 entries, 0 to 83
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Place_Id      84 non-null     int64  
 1   Place_Name    84 non-null     object 
 2   Description   84 non-null     object 
 3   Categories    84 non-null     object 
 4   City          84 non-null     object 
 5   Price (INR)   84 non-null     int64  
 6   Ratings       84 non-null     float64
 7   Time_Minutes  84 non-null     int64  
 8   Coordinate    84 non-null     object 
 9   lat           84 non-null     float64
 10  Long          84 non-null     float64
dtypes: float64(3), int64(3), object(5)
memory usage: 9.9+ KB


In [110]:
types = []
for i in df['Categories']:
  if i not in types:
    types.append(i)
  else:
    continue

print(types)

['Culture', 'Amusement park', 'Nature preserve', 'Islands', 'Shopping center', 'Religious']


In [111]:
max_budget = int(input("Enter the maximum budget of the tour, including accommodation; excluding travel within the city: "))
no_days = int(input("Enter the number of days of sightseeing: "))

In [112]:
hotel_df = pd.read_csv("hotels_jak.csv", encoding='ISO-8859-1')
hotel_df = hotel_df.drop_duplicates()
hotel_list = hotel_df['Name'].tolist()

def extract_coordinates_from_url(url):
    pattern = r"/data=!.*!3d(-?\d+\.\d+)!4d(-?\d+\.\d+)"
    match = re.search(pattern, url)

    if match:
        latitude = float(match.group(1))
        longitude = float(match.group(2))
        return latitude, longitude
    else:
        return None, None

display(hotel_df)

,Maps,Name,Rating,Type,Description,Price per Night,Amenities 1,Amenities 2,Amenities 3,Amenities 4,Latitude,Longitude
0,https://www.google.com/maps/place/Best+Western...,Best Western Senayan,4.5,3-star hotel,Casual hotel with dining & free Wi-Fi,2811,Free parking,Free breakfast,Free Wi-Fi,Air-conditioned,-6.228065,106.798835
1,https://www.google.com/maps/place/Mercure+Jaka...,Mercure Jakarta Gatot Subroto,4.6,4-star hotel,Upmarket hotel with dining & a spa,4535,Pool,Free parking,Free Wi-Fi,Air-conditioned,-6.239528,106.830420
2,https://www.google.com/maps/place/Ashley+Wahid...,Ashley Wahid Hasyim Jakarta,4.5,4-star hotel,Chic lodging with a modern restaurant,3918,Free parking,Free breakfast,Free Wi-Fi,Air-conditioned,-6.187497,106.827085
3,https://www.google.com/maps/place/HARRIS+Vertu...,HARRIS Vertu Hotel Harmoni Jakarta,4.6,4-star hotel,Cosmopolitan lodging with a spa,4007,Pool,Spa,Free parking,Free Wi-Fi,-6.164287,106.820651
4,https://www.google.com/maps/place/Hotel+Rivoli...,Hotel Rivoli Jakarta,4.2,3-star hotel,Straightforward hotel with dining,1785,Spa,Free parking,Free breakfast,Free Wi-Fi,-6.183943,106.844287
...,...,...,...,...,...,...,...,...,...,...,...,...
73,https://www.google.com/maps/place/Four+Points+...,"Four Points by Sheraton Jakarta, Thamrin",4.3,4-star hotel,Modern hotel with casual dining & a gym,4863,Spa,Free parking,Free Wi-Fi,Air-conditioned,-6.188306,106.822639
74,https://www.google.com/maps/place/K-Style+Eco+...,K-Style Eco Hotel Jakarta,3.9,2-star hotel,Modest budget hotel with a restaurant,862,Free parking,Air-conditioned,Wi-Fi,Restaurant,-6.159590,106.839515
75,https://www.google.com/maps/place/Hotel+Parago...,Hotel Paragon,4.0,3-star hotel,Casual lodging with a restaurant,1496,Free parking,Air-conditioned,Breakfast,Wi-Fi,-6.186664,106.831223
76,https://www.google.com/maps/place/THE+1O1+Jaka...,THE 1O1 Jakarta Sedayu Darmawangsa,4.5,4-star hotel,Striking hotel featuring city views,4632,Pool,Spa,Free parking,Free Wi-Fi,-6.253952,106.801820


In [113]:
def haversine_distance(lat1, lon1, lat2, lon2):
   r = 6371
   phi1 = np.radians(lat1)
   phi2 = np.radians(lat2)
   delta_phi = np.radians(lat2 - lat1)
   delta_lambda = np.radians(lon2 - lon1)
   a = np.sin(delta_phi / 2)**2 + np.cos(phi1) * np.cos(phi2) *   np.sin(delta_lambda / 2)**2
   res = r * (2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))
   ans = np.round(res, 2)
   return ans

def objective_function(solution, locations):
    total_distance = 0

    # Calculate the total distance for the solution
    for i in range(len(solution) - 1):
        current_location = solution[i]
        next_location = solution[i + 1]

        # Calculate the distance between locations
        distance = haversine_distance(locations[current_location], locations[next_location])
        total_distance += distance

    return total_distance


In [114]:
df1 = df[['Place_Name', 'lat', 'Long']]
df1.head(15)

,Place_Name,lat,Long
0,National Monument,-6.175392,106.827153
1,Old town,-6.137645,106.817125
2,Fantasy world,-6.125312,106.833538
3,Beautiful Indonesia in Miniature Park (TMII),-6.302446,106.895156
4,Atlantis Water Adventure,-6.124190,106.839134
5,Jaya Ancol Dreamland,-6.117333,106.857995
6,Ragunan Zoo,-6.312459,106.820187
7,Ocean Ecopark,-6.125802,106.836325
8,Harbor Marina,1.078880,103.931398
9,Tidung Island,-5.803205,106.523791


In [115]:
sorted_df = df[df['Ratings'] > 4]
new_df = sorted_df

categories = new_df['Categories'].unique()
print(categories)

user_categories = []
i=0
while i < len(categories):
  choice = input("Enter the categories you wish to visit (enter 0 to stop; enter 'all' to choose all categories): ")
  if choice == 'all':
    user_categories = categories
    break

  if (choice == '0'):
    break

  choice = choice.capitalize()
  if choice in categories:
    user_categories.append(choice)
    i=i+1

  else:
    print("Invalid entry")

print(user_categories)

user_categories_df = new_df[new_df['Categories'].isin(user_categories)]
df = user_categories_df
display(df)



['Culture' 'Amusement park' 'Nature preserve' 'Islands' 'Shopping center'
 'Religious']
['Culture' 'Amusement park' 'Nature preserve' 'Islands' 'Shopping center'
 'Religious']


,Place_Id,Place_Name,Description,Categories,City,Price (INR),Ratings,Time_Minutes,Coordinate,lat,Long
0,1,National Monument,The National Monument or popularly abbreviated...,Culture,Jakarta,108,4.6,15,"{'lat': -6.1753924, 'lng': 106.8271528}",-6.175392,106.827153
1,2,Old town,"The old town in Jakarta, which is also called ...",Culture,Jakarta,0,4.6,90,"{'lat': -6.137644799999999, 'lng': 106.8171245}",-6.137645,106.817125
2,3,Fantasy world,Dunia Fantasi or also called Dufan is an enter...,Amusement park,Jakarta,1458,4.6,360,"{'lat': -6.125312399999999, 'lng': 106.8335377}",-6.125312,106.833538
3,4,Beautiful Indonesia in Miniature Park (TMII),Taman Mini Indonesia Indah is a tourist park w...,Amusement park,Jakarta,54,4.5,134,"{'lat': -6.30245899999999, 'lng': 106.8951559}",-6.302446,106.895156
4,5,Atlantis Water Adventure,"Atlantis Water Adventure, also known as Atlant...",Amusement park,Jakarta,508,4.5,60,"{'lat': -6.12419, 'lng': 106.839134}",-6.124190,106.839134
...,...,...,...,...,...,...,...,...,...,...,...
79,80,Plaza Indonesia,"Plaza Indonesia was inaugurated in early 1990,...",Shopping center,Jakarta,0,4.7,120,"{'lat': -6.193925900000002, 'lng': 106.8222158}",-6.193926,106.822216
80,81,Thamrin City Mall,Thamrin City or Thamrin City Mall is a large s...,Shopping center,Jakarta,0,4.4,60,"{'lat': -6.1946096, 'lng': 106.817905}",-6.194610,106.817905
81,82,The Satria Mandala Museum,The Satria Mandala Museum is a museum of the h...,Culture,Jakarta,27,4.5,150,"{'lat': -6.231698499999999, 'lng': 106.8185425}",-6.231699,106.818543
82,83,Alive Museum Ancol,Museums now don't only offer objects of histor...,Amusement park,Jakarta,1080,4.3,134,"{'lat': -6.117534399999999, 'lng': 106.857313}",-6.117534,106.857313


In [116]:
max_tour_duration = 650  # Maximum tour duration in minutes
min_rating = 4.0
all_tours = []
TOURS = []

def get_location(place):
      lat = df.loc[df['Place_Name'] == place, 'lat'].item()
      long = df.loc[df['Place_Name'] == place, 'Long'].item()
      return lat, long

# Calculate the commute time based on the distance
def calculate_commute_time(distance):
      # Assuming a constant speed for commute
      speed = 30  # km/h
      commute_time = distance / speed  # in hours
      return (commute_time*60)

# Get the time spent at a place
def get_time_spent(place):
      time_spent = df.loc[df['Place_Name'] == place, 'Time_Minutes'].item()
      return time_spent

# Get the cost of visiting a place
def get_cost(place):
      cost = df.loc[df['Place_Name'] == place, 'Price (INR)'].item()
      return cost

# Get the rating of a place
def get_rating(place):
      rating = df.loc[df['Place_Name'] == place, 'Ratings'].item()
      return rating

# Calculate the total commute time in the tour
def calculate_total_commute_time(tour):
      total_commute_time = 0
      for i in range(len(tour)):
          place1 = tour[i]
          place2 = tour[(i + 1) % len(tour)]
          lat1, long1 = get_location(place1)
          lat2, long2 = get_location(place2)
          dist = haversine_distance(lat1, long1, lat2, long2)
          commute_time = calculate_commute_time(dist)
          total_commute_time += commute_time
      return total_commute_time


for i in range(3):
  for i in range(no_days):
    duration = []
    def initialize_population(num_moths, max_tour_duration, max_budget, min_rating):
      population = []
      attractions = df['Place_Name'].tolist()
      for _ in range(num_moths):
        tour = generate_random_tour(attractions, max_tour_duration, max_budget, min_rating)
        population.append(tour)


      return population


    def generate_random_tour(attractions, max_tour_duration, max_budget, min_rating):
      attractions = list(attractions)
      random.shuffle(attractions)
      tour = []
      distances = []
      tour_duration = 0

      if len(attractions) <= 1:
          return tour, distances

      for i in range(len(attractions)):
          attraction1 = attractions[i]

          if i < len(attractions) - 1:
              attraction2 = attractions[i + 1]

              lat1, long1 = get_location(attraction1)
              lat2, long2 = get_location(attraction2)
              dist = haversine_distance(lat1, long1, lat2, long2)

              commute_time = calculate_commute_time(dist)
              time_spent = get_time_spent(attraction2)

              # Check if adding the current attraction exceeds the max tour duration
              if tour_duration + commute_time + time_spent <= max_tour_duration:
                  distances.append([attraction1, attraction2, dist])
                  tour.append(attraction1)
                  tour_duration += commute_time + time_spent
              else:
                  # If adding the current attraction exceeds the max tour duration,
                  # calculate the remaining time available
                  remaining_time = max_tour_duration - tour_duration

                  # Check if the remaining time is enough for the commute time and time spent at the last attraction
                  if remaining_time >= (commute_time + get_time_spent(attractions[-1])):
                      # Add the last attraction to complete the tour
                      distances.append([attractions[-1], tour[0], dist])
                      tour.append(attractions[-1])
                      tour_duration += commute_time + get_time_spent(attractions[-1])
                      break  # Exit the loop since the tour is complete
                  else:
                      break  # Exit the loop since the remaining time is not enough

      # If the generated tour has less than 3 places, reshuffle and regenerate the tour

      if len(tour) < 3:
          return generate_random_tour(attractions, max_tour_duration, max_budget, min_rating)

      tour_duration = math.ceil(tour_duration)
      duration.append(tour_duration)

      return tour, distances

    tours = initialize_population(20, max_tour_duration, max_budget, min_rating)


    def calculate_tour_distance(moths):
      distances = []
      tour_list = []
      tour_distances = []
      for tour in moths:
        dists = tour[1]
        travelled_distance = 0
        for distance in dists:
          travelled_distance = travelled_distance + distance[2]
        distances.append(travelled_distance)
        tour_list.append(tour[0])

      tour_distances = [[x, y] for x, y in zip(tour_list, distances)]

      return tour_distances

    population_distances = calculate_tour_distance(tours)


    for i, j in zip(population_distances, duration):
      sum_ratings = 0
      average_rating = 0
      total_cost = 0
      for place in i[0]:
        rating_place = df.loc[df['Place_Name'] == place, 'Ratings'].iloc[0]
        cost = df.loc[df['Place_Name'] == place, 'Price (INR)'].iloc[0]
        sum_ratings = sum_ratings + rating_place
        total_cost = total_cost + cost

      average_rating = (sum_ratings) / len(i[0])
      i.append(round(average_rating, 2))
      i.append(total_cost)

      i.append(j)


    def evaluate_fitness(tours, max_tour_duration, max_budget, min_rating):
      fitness_values = []
      for tour, total_distance, total_rating, total_budget, total_duration in tours:
        if (total_duration <= max_tour_duration and total_budget <= max_budget and total_rating >= min_rating):
          fitness = calculate_fitness(total_distance, total_duration, total_budget, total_rating, max_budget, min_rating)
          fitness_values.append(fitness)
        else:
          fitness_values.append(0)  # Penalize tours violating the constraints
      return fitness_values


    def calculate_fitness(total_distance, total_duration, total_budget, total_rating, max_budget, min_rating):
      weight_distance = 1
      weight_duration = 3
      weight_budget = 2
      weight_rating = 4

      # Check if the budget constraint is violated
      if total_budget > max_budget:
        return 0  # Penalize the solution with a fitness value of 0

      # Check if the rating constraint is violated
      if total_rating < min_rating:
        return 0  # Penalize the solution with a fitness value of 0

      fitness = (
          weight_distance * (1 / total_distance)
          + weight_duration * (1 / total_duration)
          + weight_budget * ((max_budget - total_budget) / max_budget)  # Scale the budget fitness based on how close it is to the maximum budget
          + weight_rating * (total_rating - min_rating)  # Scale the rating fitness based on how far it is from the minimum rating
      )

      return fitness

    fitness = evaluate_fitness(population_distances, max_tour_duration, max_budget, min_rating)
    fitness_values = fitness


    def update_light_source(fitness, population_distances):
      best_fitness = max(fitness)
      best_tour = population_distances[fitness.index(best_fitness)][0]
      return best_tour, best_fitness

    def generate_permutations(tour, num_permutations):
      permutations = []
      for _ in range(num_permutations):
        permuted_tour = tour[:]
        random.shuffle(permuted_tour)
        permutations.append(permuted_tour)
      return permutations

    def moth_movement(moths, max_iterations, max_tour_duration, max_budget, min_rating):
      best_moth = None
      best_fitness = 0
      best_tour = []

      for iteration in range(max_iterations):
          # Evaluate fitness for each moth
          fitness = evaluate_fitness(moths, max_tour_duration, max_budget, min_rating)

          for moth, fit in zip(moths, fitness):
              # Update the best tour and fitness
              if fit > best_fitness:
                  best_moth = moth
                  best_fitness = fit
                  best_tour, best_fitness = update_light_source(fitness, moths)


          # Update the moth's position (tour) based on other moths
          other_moths = [m for m in moths if m != best_moth]
          random_moth = random.choice(other_moths)

          # Check if best_moth and random_moth have enough elements
          min_tour_length = min(len(best_moth), len(random_moth))
          for i in range(min_tour_length):
              fitness = evaluate_fitness(moths, max_tour_duration, max_budget, min_rating)
              if fitness[moths.index(random_moth)] > fitness[moths.index(best_moth)]:
                  if best_moth[i] != random_moth[i] and np.random.random() < 0.5:
                      best_moth[i] = random_moth[i]

      return best_moth


    best_tour = moth_movement(population_distances, 40, max_tour_duration, max_budget, min_rating)
    print("BEST TOUR = ",best_tour)
    all_tours.append(best_tour)
    

    sorted_fitness = sorted(enumerate(fitness_values), key=lambda x: x[1], reverse = True)
    indices = [index for index, _ in sorted_fitness]


    df = df[~df["Place_Name"].isin(best_tour[0])]

  df = user_categories_df

  my_tours = []
  for i in all_tours:
    my_tours.append(i[0])

  print("-----------------------------")

#tour, distance, rating, cost, duration

BEST TOUR =  [['Plaza Indonesia', 'Proclamation Monument', 'Macan Museum (Modern and Contemporary Art in Nusantara)', 'Old town', 'BSM Culinary Area', 'Istiqlal Mosque'], 174.06, 4.67, 1269, 540]
BEST TOUR =  [['Inscription Park Museum', 'Ahmad Yani Sasmita Loka Museum', 'Puppet Museum', 'Rich Indonesia Gallery', 'Menteng Park'], 26.879999999999995, 4.6, 49, 568]
BEST TOUR =  [['Cathedral Church', 'Spathodea Park', 'Kidzania'], 112.62, 4.58, 11, 595]
BEST TOUR =  [['Freedom Libraries', 'Ragunan Zoo', 'Pecenongan Culinary Tour', 'Basoeki Abdullah Museum', 'Puring Garden Market'], 55.68000000000001, 4.67, 28, 481]
-----------------------------
BEST TOUR =  [['Pecenongan Culinary Tour', 'Angel Island', 'Cattleya Park', 'Textile Museum', 'Set Babakan'], 68.23, 4.6, 162, 546]
BEST TOUR =  [['Atlantis Water Adventure', 'Grand Indonesia Mall', 'Beautiful Indonesia in Miniature Park (TMII)', 'Freedom Libraries', 'Bull Field Park'], 48.27, 4.68, 0, 648]
BEST TOUR =  [['State Palace of the Repub

In [117]:
for i in all_tours:
    print(i)

[['Plaza Indonesia', 'Proclamation Monument', 'Macan Museum (Modern and Contemporary Art in Nusantara)', 'Old town', 'BSM Culinary Area', 'Istiqlal Mosque'], 174.06, 4.67, 1269, 540]
[['Inscription Park Museum', 'Ahmad Yani Sasmita Loka Museum', 'Puppet Museum', 'Rich Indonesia Gallery', 'Menteng Park'], 26.879999999999995, 4.6, 49, 568]
[['Cathedral Church', 'Spathodea Park', 'Kidzania'], 112.62, 4.58, 11, 595]
[['Freedom Libraries', 'Ragunan Zoo', 'Pecenongan Culinary Tour', 'Basoeki Abdullah Museum', 'Puring Garden Market'], 55.68000000000001, 4.67, 28, 481]
[['Pecenongan Culinary Tour', 'Angel Island', 'Cattleya Park', 'Textile Museum', 'Set Babakan'], 68.23, 4.6, 162, 546]
[['Atlantis Water Adventure', 'Grand Indonesia Mall', 'Beautiful Indonesia in Miniature Park (TMII)', 'Freedom Libraries', 'Bull Field Park'], 48.27, 4.68, 0, 648]
[['State Palace of the Republic of Indonesia', 'Cibubur Garden Diary (Cibubur)', 'Rainbow Island'], 44.63, 4.63, 1010, 565]
[['Suropati Park', 'Skyri

In [118]:
cost = 0
for i in all_tours:
  cost = cost + (7000*0.0054) + (i[1]-2)*4000*0.0054

for i in my_tours:
  for j in i:
    price = final_df.loc[final_df['Place_Name'] == j, 'Price (INR)'].iloc[0]
    cost = cost + price

cost_tours = math.ceil(cost)



In [119]:
budget_constraint = max_budget - cost_tours - 500

def remove_duplicates(lst):
    seen = set()
    result = []
    for item in lst:
        place = item[0]
        if place not in seen:
            seen.add(place)
            result.append(item)
    return result

def calculate_score(distance, rating, price_per_night, total_nights, budget):
    weight_distance = 2
    weight_rating = 5
    weight_budget = 3
    total_price = price_per_night * total_nights
    budget_difference = abs(budget - total_price)
    score = (weight_distance / (distance + 0.1)) + (weight_rating * rating) + (weight_budget / (budget_difference + 0.1))
    return score

hotels = hotel_df['Name'].tolist()
hotel_ratings = hotel_df['Rating'].tolist()
hotel_prices_per_night = hotel_df['Price per Night'].tolist()
total_nights = no_days + 2  # Example number of total nights, adjust according to your requirements

hotel_scores = []
for tour in my_tours:
    first_place = tour[0]
    first_place_lat = final_df.loc[final_df['Place_Name'] == first_place, 'lat'].iloc[0]
    first_place_long = final_df.loc[final_df['Place_Name'] == first_place, 'Long'].iloc[0]

    for hotel, rating, price_per_night in zip(hotels, hotel_ratings, hotel_prices_per_night):
        url = hotel_df.loc[hotel_df['Name'] == hotel, 'Maps'].iloc[0]
        hotel_latitude, hotel_longitude = extract_coordinates_from_url(url)
        distance = haversine_distance(first_place_lat, first_place_long, hotel_latitude, hotel_longitude)
        total_price = price_per_night * total_nights
        if total_price <= budget_constraint:
            score = calculate_score(distance, rating, price_per_night, total_nights, budget_constraint)
            hotel_scores.append((hotel, rating, distance, price_per_night, score))

hotel_scores.sort(key=lambda x: (x[1], abs(x[3] * total_nights - budget_constraint), x[2]), reverse=True)  # Sort by rating (descending), total price difference (ascending), and distance (ascending)
hotel_scores = remove_duplicates(hotel_scores)

top_hotels = [hotel for hotel, _, _, _, _ in hotel_scores]
print(top_hotels)




['Hotel Des Indes, Menteng', 'YELLO Hotel Manggarai - Jakarta', 'YELLO Harmoni Jakarta', 'Swiss-Belinn Wahid Hasyim', 'Sofyan Hotel Cut Meutia Cikini Menteng Jakarta', 'Sentral Cawang Hotel Jakarta', 'Ana Hotel', 'Grand Thamrin Hotel', 'Shakti', 'Cikini Hotel', 'PAX Hotel Jakarta', 'Hotel Marcopolo Jakarta', 'Park Hotel Jakarta', 'Ã©L Hotel Royale Jakarta', 'Maple Hotel Grogol', 'P Hotel', 'Triniti Hotel Jakarta', 'Hotel Diradja', 'Hotel Rivoli Jakarta', 'The Jayakarta SP Jakarta Hotel & Spa', 'ibis budget Jakarta Cikini', 'Verse Luxe Hotel Wahid Hasyim', 'Whiz Hotel Cikini Jakarta', 'Kaisar Hotel Jakarta', 'Dreamtel Hotel Jakarta', 'Puri Mega Hotel', 'Puri Jaya Hotel', 'Hotel Paragon', 'Clay Hotel', 'K-Style Eco Hotel Jakarta', 'U Stay Hotel Gunung Sahari Jakarta', 'Smart Hotel Thamrin Jakarta']


In [120]:
desired_order = []

for i in (top_hotels):
  index = hotel_df.index[hotel_df['Name'] == i].tolist()
  desired_order.append(index[0])

# Reorder the rows based on the desired order of row labels
hotel_df_reordered = hotel_df.reindex(desired_order)


In [121]:
top_hotels = [hotel for hotel, _, _, _, _ in hotel_scores]

dff = hotel_df_reordered

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


features = ['Rating', 'Type', 'Description', 'Price per Night', 'Amenities 1', 'Amenities 2', 'Amenities 3', 'Amenities 4']

dff['Features'] = dff[features[2:]].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)

vectorizer = TfidfVectorizer()
feature_vectors = vectorizer.fit_transform(dff['Features'])

similarity_matrix = cosine_similarity(feature_vectors, feature_vectors)

def get_recommendations(keywords, top_n=5):

    user_input_doc = ' '.join(keywords)
    user_input_vector = vectorizer.transform([user_input_doc])

    similarity_scores = cosine_similarity(user_input_vector, feature_vectors)[0]

    top_indices = similarity_scores.argsort()[::-1][:top_n]
    top_hotels = dff.iloc[top_indices]

    top_hotels = top_hotels.sort_values(['Rating', 'Price per Night'], ascending=[False, True])

    return top_hotels['Name'].values

user_keywords = ['Free breakfast', 'Spa', 'Pool']
num_recommendations = 5
recommendations = get_recommendations(user_keywords, top_n=num_recommendations)

print("Recommendations based on distance, rating and budget:")

if len(top_hotels)>5:
  for i in range(5):
    print(f"{i+1}. {top_hotels[i]}")

  for hotel in top_hotels[:5]:
      recommended_hotels = dff[dff['Name'] == hotel]
      display(recommended_hotels)

else:
  for i in range(len(top_hotels)):
    print(f"{i+1}. {top_hotels[i]}")

  for hotel in top_hotels:
      recommended_hotels = dff[dff['Name'] == hotel]
      display(recommended_hotels)



print()
print(f"Recommendations based on user input keywords, rating, and price per night (Top {num_recommendations} Recommendations):")
for i, hotel in enumerate(recommendations):
    print(f"{i+1}. {hotel}")




for i, hotel in enumerate(recommendations):
    recommended_hotels = dff[dff['Name'] == hotel]
    display(recommended_hotels)




Recommendations based on distance, rating and budget:
1. Hotel Des Indes, Menteng
2. YELLO Hotel Manggarai - Jakarta
3. YELLO Harmoni Jakarta
4. Swiss-Belinn Wahid Hasyim
5. Sofyan Hotel Cut Meutia Cikini Menteng Jakarta


,Maps,Name,Rating,Type,Description,Price per Night,Amenities 1,Amenities 2,Amenities 3,Amenities 4,Latitude,Longitude,Features
52,https://www.google.com/maps/place/Hotel+Des+In...,"Hotel Des Indes, Menteng",4.6,Hotel,Upscale lodging with dining & a bar,2601,Free parking,Free Wi-Fi,Air-conditioned,Breakfast,-6.197208,106.828358,Upscale lodging with dining & a bar 2601 Free ...


,Maps,Name,Rating,Type,Description,Price per Night,Amenities 1,Amenities 2,Amenities 3,Amenities 4,Latitude,Longitude,Features
61,https://www.google.com/maps/place/YELLO+Hotel+...,YELLO Hotel Manggarai - Jakarta,4.5,3-star hotel,Funky lodging with dining & a lounge,2121,Free parking,Free Wi-Fi,Air-conditioned,Breakfast,-6.210616,106.845441,Funky lodging with dining & a lounge 2121 Free...


,Maps,Name,Rating,Type,Description,Price per Night,Amenities 1,Amenities 2,Amenities 3,Amenities 4,Latitude,Longitude,Features
50,https://www.google.com/maps/place/YELLO+Harmon...,YELLO Harmoni Jakarta,4.5,3-star hotel,Trendy lodging with a 24-hour restaurant,2486,Free parking,Free Wi-Fi,Air-conditioned,Breakfast,-6.164226,106.821258,Trendy lodging with a 24-hour restaurant 2486 ...


,Maps,Name,Rating,Type,Description,Price per Night,Amenities 1,Amenities 2,Amenities 3,Amenities 4,Latitude,Longitude,Features
77,https://www.google.com/maps/place/Swiss-Belinn...,Swiss-Belinn Wahid Hasyim,4.5,3-star hotel,Contemporary hotel with dining & a bar,2590,Spa,Free parking,Free breakfast,Free Wi-Fi,-6.18724,106.820843,Contemporary hotel with dining & a bar 2590 Sp...


...

,Maps,Name,Rating,Type,Description,Price per Night,Amenities 1,Amenities 2,Amenities 3,Amenities 4,Latitude,Longitude,Features
24,https://www.google.com/maps/place/Sentral+Cawa...,Sentral Cawang Hotel Jakarta,4.4,3-star hotel,Contemporary hotel with dining & a pool,1754,Pool,Spa,Free parking,Free breakfast,-6.246978,106.872788,Contemporary hotel with dining & a pool 1754 P...


,Maps,Name,Rating,Type,Description,Price per Night,Amenities 1,Amenities 2,Amenities 3,Amenities 4,Latitude,Longitude,Features
17,https://www.google.com/maps/place/Hotel+Marcop...,Hotel Marcopolo Jakarta,4.3,3-star hotel,Low-key lodging with a pool & dining,1988,Pool,Free parking,Free breakfast,Free Wi-Fi,-6.195442,106.837058,Low-key lodging with a pool & dining 1988 Pool...


,Maps,Name,Rating,Type,Description,Price per Night,Amenities 1,Amenities 2,Amenities 3,Amenities 4,Latitude,Longitude,Features
63,https://www.google.com/maps/place/Park+Hotel+J...,Park Hotel Jakarta,4.3,3-star hotel,Polished lodging with a bar & dining,2112,Pool,Spa,Free parking,Free Wi-Fi,-6.244127,106.875607,Polished lodging with a bar & dining 2112 Pool...


,Maps,Name,Rating,Type,Description,Price per Night,Amenities 1,Amenities 2,Amenities 3,Amenities 4,Latitude,Longitude,Features
54,https://www.google.com/maps/place/%C3%A9L+Hote...,Ã©L Hotel Royale Jakarta,4.3,4-star hotel,Stylish lodging with a pool & a spa,2568,Pool,Spa,Free parking,Free breakfast,-6.15229,106.89624,Stylish lodging with a pool & a spa 2568 Pool ...


,Maps,Name,Rating,Type,Description,Price per Night,Amenities 1,Amenities 2,Amenities 3,Amenities 4,Latitude,Longitude,Features
12,https://www.google.com/maps/place/The+Jayakart...,The Jayakarta SP Jakarta Hotel & Spa,4.2,4-star hotel,Upscale lodging with dining & a pool,1829,Pool,Spa,Free parking,Free Wi-Fi,-6.147389,106.817227,Upscale lodging with dining & a pool 1829 Pool...


In [122]:
choice = 0
my_hotel = input("Enter the hotel of your choice: ")
if my_hotel in hotel_list:
  choice = 1

while choice != 1:
  print("Invalid entry")
  my_hotel = input("Enter the hotel of your choice: ")
  if my_hotel in hotel_list:
    choice = 1


In [123]:
def get_time_spent(place):
      time_spent = final_df.loc[final_df['Place_Name'] == place, 'Time_Minutes'].item()
      return time_spent

def get_location(place):
      lat = final_df.loc[final_df['Place_Name'] == place, 'lat'].item()
      long = final_df.loc[final_df['Place_Name'] == place, 'Long'].item()
      return lat, long

def get_distance_from_hotel(place, my_hotel):
  url = hotel_df.loc[hotel_df['Name'] == my_hotel, 'Maps'].item()
  hotel_lat, hotel_long = extract_coordinates_from_url(url)
  lat1, long1 = get_location(place)
  dist = haversine_distance(hotel_lat,hotel_long,lat1,long1)
  return dist

def generate_itinerary(places):
    itinerary = []
    current_time = 9 * 60  # Starting time in minutes (9 AM = 9 * 60 minutes)

    # Add hotel as the first place in the itinerary
    hotel = my_hotel

    for i in range(len(places)):
        place = places[i]
        time_spent = get_time_spent(place)

        if i == 0:
            commute_time = calculate_commute_time(get_distance_from_hotel(place, my_hotel))  # Calculate commute time from hotel
        else:
            previous_place = places[i - 1]
            lat1, long1 = get_location(previous_place)
            lat2, long2 = get_location(place)
            dist = haversine_distance(lat1, long1, lat2, long2)
            commute_time = calculate_commute_time(dist)

        start_time = current_time + commute_time
        end_time = start_time + time_spent

        itinerary.append((place, start_time, end_time))

        current_time = end_time

    return itinerary


In [124]:
TOURS = all_tours
#print(TOURS)
TOTAL_COST = 0
hotel_price = hotel_df.loc[hotel_df['Name'] == my_hotel, 'Price per Night'].iloc[0]

print(f"Here's your complete itinerary 1: Hotel - {my_hotel} - Price per Night = Rs. {hotel_price}\n")
for i in range(no_days):
  lat = final_df.loc[final_df["Place_Name"] == TOURS[i][0][0], "lat"].item()
  Long = final_df.loc[final_df["Place_Name"] == TOURS[i][0][0], "Long"].item()
  hotel_index = hotel_list.index(my_hotel)
  hotel_lat, hotel_long = extract_coordinates_from_url(hotel_df.loc[hotel_index, 'Maps'])
  dist = haversine_distance(hotel_lat, hotel_long, lat, Long)

  lat2 = final_df.loc[final_df["Place_Name"] == TOURS[i][0][-1], "lat"].item()
  Long2 = final_df.loc[final_df["Place_Name"] == TOURS[i][0][-1], "Long"].item()
  dist2 = haversine_distance(hotel_lat, hotel_long, lat2, Long2)

  TOTAL_DISTANCE = TOURS[i][1] + dist + dist2
  TOTAL_DISTANCE = round(TOTAL_DISTANCE, 2)

  total_cab_cost = (7000*0.0054) + ((TOTAL_DISTANCE - 2)*4000*0.0054)
  total_cab_cost = math.ceil(total_cab_cost)

  print(f"Day {i+1} - ")
  print("Tour -",TOURS[i][0])
  print("Total distance (Starting from and ending at hotel) -",TOTAL_DISTANCE,"km")
  print("Rating - ",TOURS[i][2])
  print("Cost - Rs.",TOURS[i][3])
  print("Taxi cost - Rs.",total_cab_cost)
  print("----------------------------------")

  TOTAL_COST = TOTAL_COST + TOURS[i][3] + total_cab_cost



print("The cost for the whole trip is Rs.", TOTAL_COST)

print(f"Hotel cost = Rs. {(no_days+2)*hotel_price} for {no_days+2} nights.")
print(f"Total cost = Rs. {(no_days+2)*hotel_price + TOTAL_COST}")


Here's your complete itinerary 1: Hotel - Erian Hotel - Price per Night = Rs. 2842

Day 1 - 
Tour - ['Plaza Indonesia', 'Proclamation Monument', 'Macan Museum (Modern and Contemporary Art in Nusantara)', 'Old town', 'BSM Culinary Area', 'Istiqlal Mosque']
Total distance (Starting from and ending at hotel) - 177.05 km
Rating -  4.67
Cost - Rs. 1269
Taxi cost - Rs. 3819
----------------------------------
Day 2 - 
Tour - ['Inscription Park Museum', 'Ahmad Yani Sasmita Loka Museum', 'Puppet Museum', 'Rich Indonesia Gallery', 'Menteng Park']
Total distance (Starting from and ending at hotel) - 29.91 km
Rating -  4.6
Cost - Rs. 49
Taxi cost - Rs. 641
----------------------------------
Day 3 - 
Tour - ['Cathedral Church', 'Spathodea Park', 'Kidzania']
Total distance (Starting from and ending at hotel) - 119.39 km
Rating -  4.58
Cost - Rs. 11
Taxi cost - Rs. 2574
----------------------------------
Day 4 - 
Tour - ['Freedom Libraries', 'Ragunan Zoo', 'Pecenongan Culinary Tour', 'Basoeki Abdulla

In [125]:
print("Detailed Itinerary:")
print("Hotel: ",my_hotel)
print()
for i in all_tours[:4]:
    places = i[0]
    itinerary = generate_itinerary(places)

    print("Start from hotel at 09:00")
    for item in itinerary:
        place = item[0]
        start_time = divmod(item[1], 60)
        end_time = divmod(item[2], 60)  # Convert minutes to hours and minutes
        start_time_str = f"{int(start_time[0]):02}:{int(start_time[1]):02}"
        end_time_str = f"{int(end_time[0]):02}:{int(end_time[1]):02}"
        print(f"{start_time_str} to {end_time_str} - {place}")

    print("--------------------------------------------------------------")




Detailed Itinerary:
Hotel:  Erian Hotel

Start from hotel at 09:00
09:02 to 11:02 - Plaza Indonesia
11:07 to 12:42 - Proclamation Monument
13:00 to 15:00 - Macan Museum (Modern and Contemporary Art in Nusantara)
15:16 to 16:46 - Old town
16:57 to 18:07 - BSM Culinary Area
18:10 to 18:55 - Istiqlal Mosque
--------------------------------------------------------------
Start from hotel at 09:00
09:04 to 10:34 - Inscription Park Museum
10:42 to 11:27 - Ahmad Yani Sasmita Loka Museum
11:43 to 14:13 - Puppet Museum
14:27 to 15:57 - Rich Indonesia Gallery
15:59 to 18:13 - Menteng Park
--------------------------------------------------------------
Start from hotel at 09:00
09:04 to 09:49 - Cathedral Church
10:23 to 10:53 - Spathodea Park
11:15 to 16:15 - Kidzania
--------------------------------------------------------------
Start from hotel at 09:00
09:04 to 10:39 - Freedom Libraries
11:04 to 11:19 - Ragunan Zoo
11:52 to 13:02 - Pecenongan Culinary Tour
13:30 to 15:00 - Basoeki Abdullah Museu

In [126]:
for i in range(4):
  temp = all_tours.pop(0)

TOURS = all_tours

TOTAL_COST = 0
hotel_price = hotel_df.loc[hotel_df['Name'] == my_hotel, 'Price per Night'].iloc[0]

print(f"Here's your complete itinerary 2: Hotel - {my_hotel} - Price per Night = Rs. {hotel_price}\n")
for i in range(no_days):
  lat = final_df.loc[final_df["Place_Name"] == TOURS[i][0][0], "lat"].item()
  Long = final_df.loc[final_df["Place_Name"] == TOURS[i][0][0], "Long"].item()
  hotel_index = hotel_list.index(my_hotel)
  hotel_lat, hotel_long = extract_coordinates_from_url(hotel_df.loc[hotel_index, 'Maps'])
  dist = haversine_distance(hotel_lat, hotel_long, lat, Long)

  lat2 = final_df.loc[final_df["Place_Name"] == TOURS[i][0][-1], "lat"].item()
  Long2 = final_df.loc[final_df["Place_Name"] == TOURS[i][0][-1], "Long"].item()
  dist2 = haversine_distance(hotel_lat, hotel_long, lat2, Long2)

  TOTAL_DISTANCE = TOURS[i][1] + dist + dist2
  TOTAL_DISTANCE = round(TOTAL_DISTANCE, 2)

  total_cab_cost = (7000*0.0054) + ((TOTAL_DISTANCE - 2)*4000*0.0054)
  total_cab_cost = math.ceil(total_cab_cost)

  print(f"Day {i+1} - ")
  print("Tour -",TOURS[i][0])
  print("Total distance (Starting from and ending at hotel) -",TOTAL_DISTANCE,"km")
  print("Rating - ",TOURS[i][2])
  print("Cost - Rs.",TOURS[i][3])
  print("Taxi cost - Rs.",total_cab_cost)
  print("----------------------------------")

  TOTAL_COST = TOTAL_COST + TOURS[i][3] + total_cab_cost



print("The cost for the whole trip is Rs.", TOTAL_COST)

print(f"Hotel cost = Rs. {(no_days+2)*hotel_price} for {no_days+2} nights.")
print(f"Total cost = Rs. {(no_days+2)*hotel_price + TOTAL_COST}")


Here's your complete itinerary 2: Hotel - Erian Hotel - Price per Night = Rs. 2842

Day 1 - 
Tour - ['Pecenongan Culinary Tour', 'Angel Island', 'Cattleya Park', 'Textile Museum', 'Set Babakan']
Total distance (Starting from and ending at hotel) - 87.71 km
Rating -  4.6
Cost - Rs. 162
Taxi cost - Rs. 1890
----------------------------------
Day 2 - 
Tour - ['Atlantis Water Adventure', 'Grand Indonesia Mall', 'Beautiful Indonesia in Miniature Park (TMII)', 'Freedom Libraries', 'Bull Field Park']
Total distance (Starting from and ending at hotel) - 57.38 km
Rating -  4.68
Cost - Rs. 0
Taxi cost - Rs. 1235
----------------------------------
Day 3 - 
Tour - ['State Palace of the Republic of Indonesia', 'Cibubur Garden Diary (Cibubur)', 'Rainbow Island']
Total distance (Starting from and ending at hotel) - 118.73 km
Rating -  4.63
Cost - Rs. 1010
Taxi cost - Rs. 2560
----------------------------------
Day 4 - 
Tour - ['Suropati Park', 'Skyrink - Taman Anggrek Mall', 'Indonesian Bank Museum']

In [127]:
print("Detailed Itinerary:")
print("Hotel: ",my_hotel)
print()
for i in all_tours[:4]:
    places = i[0]
    itinerary = generate_itinerary(places)

    print("Start from hotel at 09:00")
    for item in itinerary:
        place = item[0]
        start_time = divmod(item[1], 60)
        end_time = divmod(item[2], 60)  # Convert minutes to hours and minutes
        start_time_str = f"{int(start_time[0]):02}:{int(start_time[1]):02}"
        end_time_str = f"{int(end_time[0]):02}:{int(end_time[1]):02}"
        print(f"{start_time_str} to {end_time_str} - {place}")

    print("--------------------------------------------------------------")




Detailed Itinerary:
Hotel:  Erian Hotel

Start from hotel at 09:00
09:04 to 10:14 - Pecenongan Culinary Tour
10:48 to 12:18 - Angel Island
12:52 to 15:06 - Cattleya Park
15:10 to 15:55 - Textile Museum
16:29 to 18:04 - Set Babakan
--------------------------------------------------------------
Start from hotel at 09:00
09:14 to 10:14 - Atlantis Water Adventure
10:30 to 12:30 - Grand Indonesia Mall
12:59 to 15:13 - Beautiful Indonesia in Miniature Park (TMII)
15:38 to 17:13 - Freedom Libraries
17:20 to 19:34 - Bull Field Park
--------------------------------------------------------------
Start from hotel at 09:00
09:04 to 10:39 - State Palace of the Republic of Indonesia
11:29 to 11:44 - Cibubur Garden Diary (Cibubur)
14:53 to 16:23 - Rainbow Island
--------------------------------------------------------------
Start from hotel at 09:00
09:02 to 11:16 - Suropati Park
11:26 to 14:26 - Skyrink - Taman Anggrek Mall
14:37 to 16:12 - Indonesian Bank Museum
------------------------------------

In [128]:
for i in range(4):
  temp = all_tours.pop(0)

TOURS = all_tours

TOTAL_COST = 0
hotel_price = hotel_df.loc[hotel_df['Name'] == my_hotel, 'Price per Night'].iloc[0]

print(f"Here's your complete itinerary 3: Hotel - {my_hotel} - Price per Night = Rs. {hotel_price}\n")
for i in range(no_days):
  lat = final_df.loc[final_df["Place_Name"] == TOURS[i][0][0], "lat"].item()
  Long = final_df.loc[final_df["Place_Name"] == TOURS[i][0][0], "Long"].item()
  hotel_index = hotel_list.index(my_hotel)
  hotel_lat, hotel_long = extract_coordinates_from_url(hotel_df.loc[hotel_index, 'Maps'])
  dist = haversine_distance(hotel_lat, hotel_long, lat, Long)

  lat2 = final_df.loc[final_df["Place_Name"] == TOURS[i][0][-1], "lat"].item()
  Long2 = final_df.loc[final_df["Place_Name"] == TOURS[i][0][-1], "Long"].item()
  dist2 = haversine_distance(hotel_lat, hotel_long, lat2, Long2)

  TOTAL_DISTANCE = TOURS[i][1] + dist + dist2
  TOTAL_DISTANCE = round(TOTAL_DISTANCE, 2)

  total_cab_cost = (7000*0.0054) + ((TOTAL_DISTANCE - 2)*4000*0.0054)
  total_cab_cost = math.ceil(total_cab_cost)

  print(f"Day {i+1} - ")
  print("Tour -",TOURS[i][0])
  print("Total distance (Starting from and ending at hotel) -",TOTAL_DISTANCE,"km")
  print("Rating - ",TOURS[i][2])
  print("Cost - Rs.",TOURS[i][3])
  print("Taxi cost - Rs.",total_cab_cost)
  print("----------------------------------")

  TOTAL_COST = TOTAL_COST + TOURS[i][3] + total_cab_cost



print("The cost for the whole trip is Rs.", TOTAL_COST)

print(f"Hotel cost = Rs. {(no_days+2)*hotel_price} for {no_days+2} nights.")
print(f"Total cost = Rs. {(no_days+2)*hotel_price + TOTAL_COST}")


Here's your complete itinerary 3: Hotel - Erian Hotel - Price per Night = Rs. 2842

Day 1 - 
Tour - ['Art market', 'Pecenongan Culinary Tour', 'Cathedral Church', 'Kidzania']
Total distance (Starting from and ending at hotel) - 35.69 km
Rating -  4.7
Cost - Rs. 999
Taxi cost - Rs. 766
----------------------------------
Day 2 - 
Tour - ['Ismail Marzuki Park', 'Suropati Park', 'Rainbow Island', 'Ragunan Zoo']
Total distance (Starting from and ending at hotel) - 161.11 km
Rating -  4.62
Cost - Rs. 11
Taxi cost - Rs. 3475
----------------------------------
Day 3 - 
Tour - ['Istiqlal Mosque', 'Ahmad Yani Sasmita Loka Museum', 'National Museum', 'Jakarta Aquarium and Safari', 'Situ Lembang Park', 'Inscription Park Museum']
Total distance (Starting from and ending at hotel) - 27.58 km
Rating -  4.6
Cost - Rs. 508
Taxi cost - Rs. 591
----------------------------------
Day 4 - 
Tour - ['National Awakening Museum', 'Beautiful Indonesia in Miniature Park (TMII)', 'Bull Field Park']
Total distance

In [129]:
print("Detailed Itinerary:")
print("Hotel: ",my_hotel)
print()
for i in all_tours[:4]:
    places = i[0]
    itinerary = generate_itinerary(places)

    print("Start from hotel at 09:00")
    for item in itinerary:
        place = item[0]
        start_time = divmod(item[1], 60)
        end_time = divmod(item[2], 60)  # Convert minutes to hours and minutes
        start_time_str = f"{int(start_time[0]):02}:{int(start_time[1]):02}"
        end_time_str = f"{int(end_time[0]):02}:{int(end_time[1]):02}"
        print(f"{start_time_str} to {end_time_str} - {place}")

    print("--------------------------------------------------------------")




Detailed Itinerary:
Hotel:  Erian Hotel

Start from hotel at 09:00
09:03 to 10:33 - Art market
10:37 to 11:47 - Pecenongan Culinary Tour
11:49 to 12:34 - Cathedral Church
12:47 to 17:47 - Kidzania
--------------------------------------------------------------
Start from hotel at 09:00
09:02 to 10:37 - Ismail Marzuki Park
10:40 to 12:54 - Suropati Park
15:20 to 16:50 - Rainbow Island
19:39 to 19:54 - Ragunan Zoo
--------------------------------------------------------------
Start from hotel at 09:00
09:03 to 09:48 - Istiqlal Mosque
09:56 to 10:41 - Ahmad Yani Sasmita Loka Museum
10:48 to 12:23 - National Museum
12:30 to 14:44 - Jakarta Aquarium and Safari
14:55 to 17:09 - Situ Lembang Park
17:16 to 18:46 - Inscription Park Museum
--------------------------------------------------------------
Start from hotel at 09:00
09:02 to 09:47 - National Awakening Museum
10:18 to 12:32 - Beautiful Indonesia in Miniature Park (TMII)
13:04 to 15:18 - Bull Field Park
----------------------------------